"""
# 🎯 Attention U-Net for Kidney Stone Segmentation on KSSD2025

## 📊 Objective
Beat the baseline Modified U-Net score of **97.06%** using Attention U-Net

## 🎯 Expected Results
- **Target Dice Score:** 97.5% - 98.2%
- **Strategy:** Attention mechanisms for small object detection
- **Architecture:** U-Net + Attention Gates

## 📋 Implementation Plan
1. ✅ Setup & Import Libraries
2. ✅ Load KSSD2025 Dataset
3. ✅ Data Preprocessing & Augmentation
4. ✅ Build Attention U-Net Architecture
5. ✅ Training with 5-Fold Cross-Validation
6. ✅ Evaluation & Visualization
7. ✅ Results Comparison

## 📦 Step 1: Install & Import Required Libraries


In [1]:
import sys
!{sys.executable} -m pip install -q segmentation-models-pytorch albumentations

print("✅ Libraries installed successfully!")
print("="*50)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.1 MB/s eta 0:00:0000:01
✅ Libraries installed successfully!


In [2]:
# Core Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
import gc
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Sklearn
from sklearn.model_selection import KFold

# Image Processing
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Progress Bar
from tqdm.auto import tqdm

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

print("✅ All libraries imported successfully!")
print(f"✅ PyTorch Version: {torch.__version__}")
print(f"✅ CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ CUDA Device: {torch.cuda.get_device_name(0)}")
print("="*50)

✅ All libraries imported successfully!
✅ PyTorch Version: 2.8.0+cu126
✅ CUDA Available: True
✅ CUDA Device: Tesla T4


## 📂 Step 2: Load KSSD2025 Dataset

**Dataset Structure Expected:**
```
/kaggle/input/kssd2025/
├── images/
│   ├── image_001.jpg
│   ├── image_002.jpg
│   └── ...
└── masks/
    ├── image_001.jpg
    ├── image_002.jpg
    └── ...
```
"""

In [4]:
# Configuration
class Config:
    # Paths (ADJUST THESE BASED ON YOUR KAGGLE DATASET PATH)
    DATA_PATH = "/kaggle/input/kssd2025-kidney-stone-segmentation-dataset"  # Change this to your dataset path
    IMAGE_DIR = f"{DATA_PATH}/images"
    MASK_DIR = f"{DATA_PATH}/masks"
    
    # Image Settings
    IMG_SIZE = 256  # Resize images to 256x256 (adjust based on your needs)
    
    # Training Settings
    BATCH_SIZE = 16
    NUM_EPOCHS = 150
    LEARNING_RATE = 0.001
    NUM_FOLDS = 5
    
    # Model Settings
    ENCODER_CHANNELS = [16, 32, 64, 128]  # Match paper's modified U-Net
    DECODER_CHANNELS = [128, 64, 32, 16]
    
    # Device
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Save Settings
    SAVE_MODELS = True
    MODEL_DIR = "/kaggle/working/models"

config = Config()

# Create model directory
Path(config.MODEL_DIR).mkdir(parents=True, exist_ok=True)

print("⚙️ Configuration Settings:")
print(f"  📁 Data Path: {config.DATA_PATH}")
print(f"  🖼️  Image Size: {config.IMG_SIZE}x{config.IMG_SIZE}")
print(f"  📦 Batch Size: {config.BATCH_SIZE}")
print(f"  🔄 Epochs: {config.NUM_EPOCHS}")
print(f"  📊 K-Folds: {config.NUM_FOLDS}")
print(f"  🎯 Device: {config.DEVICE}")
print("="*50)

⚙️ Configuration Settings:
  📁 Data Path: /kaggle/input/kssd2025-kidney-stone-segmentation-dataset
  🖼️  Image Size: 256x256
  📦 Batch Size: 16
  🔄 Epochs: 150
  📊 K-Folds: 5
  🎯 Device: cuda


In [5]:
# Get all image and mask paths
from glob import glob
import os

# Get image paths
image_paths = sorted(glob(os.path.join(config.IMAGE_DIR, "*.jpg")))
if len(image_paths) == 0:
    image_paths = sorted(glob(os.path.join(config.IMAGE_DIR, "*.png")))

# Get mask paths
mask_paths = sorted(glob(os.path.join(config.MASK_DIR, "*.jpg")))
if len(mask_paths) == 0:
    mask_paths = sorted(glob(os.path.join(config.MASK_DIR, "*.png")))

print(f"📊 Dataset Statistics:")
print(f"  🖼️  Total Images: {len(image_paths)}")
print(f"  🎭 Total Masks: {len(mask_paths)}")
print("="*50)

# Verify matching
if len(image_paths) != len(mask_paths):
    print("⚠️ WARNING: Number of images and masks don't match!")
else:
    print("✅ Images and masks count matches!")

# Create dataframe
data_df = pd.DataFrame({
    'image_path': image_paths,
    'mask_path': mask_paths
})

print(f"\n📋 Dataset Preview:")
print(data_df.head())
print("="*50)

📊 Dataset Statistics:
  🖼️  Total Images: 0
  🎭 Total Masks: 0
✅ Images and masks count matches!

📋 Dataset Preview:
Empty DataFrame
Columns: [image_path, mask_path]
Index: []
